In [111]:
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup
import os
import csv
import itertools
import re

In [162]:
page = open("/Users/yves/Desktop/Data_Science/first_year/first_semester/adm/adm_hw3/html_files/movie5.html", "r")
page = page.read()

In [163]:
f =  BeautifulSoup(page)

In [166]:

big_page = 'https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies1.html'
big_page_response = requests.get(big_page)

big_page_soup = BeautifulSoup(big_page_response.text, 'html.parser')

url_list = []
for i in big_page_soup.find_all('tr')[1:]:
    td = i.find_all('a')[0].get("href") # get the url in the second column of each row starting from the sec. row
    url_list.append(td)

# questo mi derve per pulire gli output dell'infobox
def clean_col(string): 
    clean1 = string.strip('\n') # remove all escape characters
    pattern = r'\[.*?\]'  # remove all square brakets and content
    clean2 = re.sub(pattern, '', clean1)
    return(clean2)

# questo per ricavare i dati dall'infobox
def info_tsv(film_soup, movie_number):
    
    pattern = r'\[.*?\]'  # remove all square brakets and content
    
    infb = ['Directed by', 'Produced by', 'Written by', 'Screenplay by',
           'Story by', 'Based on', 'Starring', 'Narrated by', 'Music by',
           'Cinematography', 'Edited by', 'Production company', 
            'Distributed by', 'Release date', 'Running time', 'Country',
           'Language', 'Budget'] # lista degli elementi che vogliamo
                                # ho aagiunto anche cose inutili sempre 
                                # per provare il codice
    L = []  # Questa lista conterrà tutti gli elementi trovati nell'infob.
            # e quelli mancanti. 
        
    # Retrieve the Title   
        
    Initial_Title = film_soup.select("#firstHeading")[0].text
    L.append(['Title', Initial_Title])
    
    
    # Retrieve the Intro
    
    contents = film_soup.findAll('div', attrs={'id': 'toc'}) 
    if len(contents) > 0:
        Intr = contents[0].fetchPreviousSiblings('p')
        Intro = []
        if len(Intr) > 0:
            for p in reversed(Intr):
                if len(p.text) > 5:
                    Intro.append(p.text)
                else:
                    continue
            Intro = ''.join(Intro)
            Final_intro = re.sub(pattern, '', Intro)
            if len(Final_intro)>20:
                L.append(['Intro', Final_intro])
            else:
                L.append(['Intro', 'NA'])
    
    
    # Retrieve the Plot
    
    plot = []
    possibles = ['Plot','Synopsis','Plot synopsis','Plot summary', 
                 'Story','Plotline','The Beginning','Summary',
                'Content','Premise', 'Intro', 'Intro']
    for i in possibles:
        # find the node with id of "Plot"
        items = film_soup.find(id=i)
        if items is None or len(items) == 0:
            continue
        else:
            # walk through the siblings of the parent (H2) node 
            # until we reach the next H2 node
            for element in items.parent.nextSiblingGenerator():
                if element.name == "h2":
                    break
                if hasattr(element, "text"):
                    plot.append(element.text)
    plot = "".join(plot)
    # remove all square brakets and content
    Final_plot = re.sub(pattern, '', plot)
    if len(Final_plot) > 20:
        L.append(['Plot', Final_plot])
    else:
        L.append(['Plot', 'NA'])
    
    # Retrieve info in the infobox
        
    if len(film_soup.findAll('table'))>0:
        if len(film_soup.findAll('table')[0].tbody.findAll('tr'))>0:
            Title = film_soup.findAll('table')[0].tbody.findAll('tr')[0].text
            L.append(['Name', Title]) # mi ricavo il titolo dall'infobox
        else:
            L.append(['Name', 'NA'])
            
    in_table = [] # mi savo gli elementi dell'infobox qui
    
    if len(film_soup.findAll('table'))>0:
        if len(film_soup.findAll('table')[0].tbody.findAll('tr'))>0:
            for row in film_soup.findAll('table')[0].tbody.findAll('tr')[1:]:
                first_col = row.findAll('th') # first column of infobox
                if len(first_col)>0:
                    second_col = row.findAll('td')
                    if len(second_col)>0:
                        second_col = row.findAll('td')[0].text # second column of info
                        new_second_col = clean_col(second_col) # clean the output

                        in_table.append(first_col[0].text) 
                        in_table.append(new_second_col)
                        

            # put NA to all missing values
            for i in infb:
                if i in in_table:
                    index = in_table.index(i)
                    L.append([i, in_table[index +1]])
                else:
                    L.append([i, 'NA']) 
    L.append(['Url', url_list[int(movie_number) -1]])
    return(L)

In [167]:
# here is an example
table= info_tsv(f,5)
print(url_list[4])
df = pd.DataFrame(table).T
df.columns = df.iloc[0]
df = df.drop(df.index[0])
table

https://en.wikipedia.org/wiki/Alice_in_Wonderland_(1903_film)


,Title,Intro,Plot,Name,Directed by,Produced by,Written by,Screenplay by,Story by,Based on,...,Cinematography,Edited by,Production company,Distributed by,Release date,Running time,Country,Language,Budget,Url
1,Alice in Wonderland (1903 film),Alice in Wonderland is a 1903 British silent f...,"Alice follows a large white rabbit down a ""Rab...",Alice in Wonderland,Cecil HepworthPercy Stow,Cecil M. HepworthHerman Casler (exec. producer...,Lewis Carroll (book)Cecil M. Hepworth,NA,NA,Alice's Adventures in Wonderland and Through t...,...,Cecil M. Hepworth,NA,NA,American Mutoscope and Biograph CompanyEdison ...,17 October 1903 (1903-10-17),approx. 8:19 minutes,United Kingdom,Silent,NA,https://en.wikipedia.org/wiki/Alice_in_Wonderl...


In [168]:
movies_done = []

In [169]:
# This is to create a single tsv file
# by addiing two lines we could create a tsv for each movie
    

paths = '/Users/yves/Desktop/Data_Science/first_year/first_semester/adm/adm_hw3/html_files/'

    
for filename in os.listdir(paths):
    if filename not in movies_done:
        if filename.endswith(".html"):
            ff = open(paths + filename, 'r')
            html = ff.read()
            film = BeautifulSoup(html)

            page_name = filename.rstrip('.html') # without extension
            movies_done.append(filename)
            
            movie_number = re.findall('\d+', filename )
            movie_number = movie_number[0]
            
            with open('/Users/yves/Desktop/Data_Science/first_year/first_semester/adm/adm_hw3/tsv_files/'+page_name+'.tsv', 'wt') as out_file:
                tsv_writer = csv.writer(out_file, delimiter='\t')
                if info_tsv(film,movie_number ) is None or len(info_tsv(film, movie_number)) == 0:
                    continue
                else:
                    for col in info_tsv(film, movie_number):
                        tsv_writer.writerow([col[0], col[1]])
            print('tsv created '+ str(len(movies_done)))

            continue

        else:
            continue
    else:
        continue
print('DONE')




tsv created 1
tsv created 2
tsv created 3
tsv created 4
tsv created 5
tsv created 6
tsv created 7
tsv created 8
tsv created 9
tsv created 10
tsv created 11
tsv created 12
tsv created 13
tsv created 14
tsv created 15
tsv created 16
tsv created 17
tsv created 18
tsv created 19
tsv created 20
tsv created 21
tsv created 22
tsv created 23
tsv created 24
tsv created 25
tsv created 26
tsv created 27
tsv created 28
tsv created 29
tsv created 30
tsv created 31
tsv created 32
tsv created 33
tsv created 34
tsv created 35
tsv created 36
tsv created 37
tsv created 38
tsv created 39
tsv created 40
tsv created 41
tsv created 42
tsv created 43
tsv created 44
tsv created 45
tsv created 46
tsv created 47
tsv created 48
tsv created 49
tsv created 50
tsv created 51
tsv created 52
tsv created 53
tsv created 54
tsv created 55
tsv created 56
tsv created 57
tsv created 58
tsv created 59
tsv created 60
tsv created 61
tsv created 62
tsv created 63
tsv created 64
tsv created 65
tsv created 66
tsv created 67
tsv 

tsv created 520
tsv created 521
tsv created 522
tsv created 523
tsv created 524
tsv created 525
tsv created 526
tsv created 527
tsv created 528
tsv created 529
tsv created 530
tsv created 531
tsv created 532
tsv created 533
tsv created 534
tsv created 535
tsv created 536
tsv created 537
tsv created 538
tsv created 539
tsv created 540
tsv created 541
tsv created 542
tsv created 543
tsv created 544
tsv created 545
tsv created 546
tsv created 547
tsv created 548
tsv created 549
tsv created 550
tsv created 551
tsv created 552
tsv created 553
tsv created 554
tsv created 555
tsv created 556
tsv created 557
tsv created 558
tsv created 559
tsv created 560
tsv created 561
tsv created 562
tsv created 563
tsv created 564
tsv created 565
tsv created 566
tsv created 567
tsv created 568
tsv created 569
tsv created 570
tsv created 571
tsv created 572
tsv created 573
tsv created 574
tsv created 575
tsv created 576
tsv created 577
tsv created 578
tsv created 579
tsv created 580
tsv created 581
tsv crea

tsv created 1031
tsv created 1032
tsv created 1033
tsv created 1034
tsv created 1035
tsv created 1036
tsv created 1037
tsv created 1038
tsv created 1039
tsv created 1040
tsv created 1041
tsv created 1042
tsv created 1043
tsv created 1044
tsv created 1045
tsv created 1046
tsv created 1047
tsv created 1048
tsv created 1049
tsv created 1050
tsv created 1051
tsv created 1052
tsv created 1053
tsv created 1054
tsv created 1055
tsv created 1056
tsv created 1057
tsv created 1058
tsv created 1059
tsv created 1060
tsv created 1061
tsv created 1062
tsv created 1063
tsv created 1064
tsv created 1065
tsv created 1066
tsv created 1067
tsv created 1068
tsv created 1069
tsv created 1070
tsv created 1071
tsv created 1072
tsv created 1073
tsv created 1074
tsv created 1075
tsv created 1076
tsv created 1077
tsv created 1078
tsv created 1079
tsv created 1080
tsv created 1081
tsv created 1082
tsv created 1083
tsv created 1084
tsv created 1085
tsv created 1086
tsv created 1087
tsv created 1088
tsv created 10

tsv created 1513
tsv created 1514
tsv created 1515
tsv created 1516
tsv created 1517
tsv created 1518
tsv created 1519
tsv created 1520
tsv created 1521
tsv created 1522
tsv created 1523
tsv created 1524
tsv created 1525
tsv created 1526
tsv created 1527
tsv created 1528
tsv created 1529
tsv created 1530
tsv created 1531
tsv created 1532
tsv created 1533
tsv created 1534
tsv created 1535
tsv created 1536
tsv created 1537
tsv created 1538
tsv created 1539
tsv created 1540
tsv created 1541
tsv created 1542
tsv created 1543
tsv created 1544
tsv created 1545
tsv created 1546
tsv created 1547
tsv created 1548
tsv created 1549
tsv created 1550
tsv created 1551
tsv created 1552
tsv created 1553
tsv created 1554
tsv created 1555
tsv created 1556
tsv created 1557
tsv created 1558
tsv created 1559
tsv created 1560
tsv created 1561
tsv created 1562
tsv created 1563
tsv created 1564
tsv created 1565
tsv created 1566
tsv created 1567
tsv created 1568
tsv created 1569
tsv created 1570
tsv created 15

tsv created 1996
tsv created 1997
tsv created 1998
tsv created 1999
tsv created 2000
tsv created 2001
tsv created 2002
tsv created 2003
tsv created 2004
tsv created 2005
tsv created 2006
tsv created 2007
tsv created 2008
tsv created 2009
tsv created 2010
tsv created 2011
tsv created 2012
tsv created 2013
tsv created 2014
tsv created 2015
tsv created 2016
tsv created 2017
tsv created 2018
tsv created 2019
tsv created 2020
tsv created 2021
tsv created 2022
tsv created 2023
tsv created 2024
tsv created 2025
tsv created 2026
tsv created 2027
tsv created 2028
tsv created 2029
tsv created 2030
tsv created 2031
tsv created 2032
tsv created 2033
tsv created 2034
tsv created 2035
tsv created 2036
tsv created 2037
tsv created 2038
tsv created 2039
tsv created 2040
tsv created 2041
tsv created 2042
tsv created 2043
tsv created 2044
tsv created 2045
tsv created 2046
tsv created 2047
tsv created 2048
tsv created 2049
tsv created 2050
tsv created 2051
tsv created 2052
tsv created 2053
tsv created 20

...

tsv created 7783
tsv created 7784
tsv created 7785
tsv created 7786
tsv created 7787
tsv created 7788
tsv created 7789
tsv created 7790
tsv created 7791
tsv created 7792
tsv created 7793
tsv created 7794
tsv created 7795
tsv created 7796
tsv created 7797
tsv created 7798
tsv created 7799
tsv created 7800
tsv created 7801
tsv created 7802
tsv created 7803
tsv created 7804
tsv created 7805
tsv created 7806
tsv created 7807
tsv created 7808
tsv created 7809
tsv created 7810
tsv created 7811
tsv created 7812
tsv created 7813
tsv created 7814
tsv created 7815
tsv created 7816
tsv created 7817
tsv created 7818
tsv created 7819
tsv created 7820
tsv created 7821
tsv created 7822
tsv created 7823
tsv created 7824
tsv created 7825
tsv created 7826
tsv created 7827
tsv created 7828
tsv created 7829
tsv created 7830
tsv created 7831
tsv created 7832
tsv created 7833
tsv created 7834
tsv created 7835
tsv created 7836
tsv created 7837
tsv created 7838
tsv created 7839
tsv created 7840
tsv created 78

tsv created 8265
tsv created 8266
tsv created 8267
tsv created 8268
tsv created 8269
tsv created 8270
tsv created 8271
tsv created 8272
tsv created 8273
tsv created 8274
tsv created 8275
tsv created 8276
tsv created 8277
tsv created 8278
tsv created 8279
tsv created 8280
tsv created 8281
tsv created 8282
tsv created 8283
tsv created 8284
tsv created 8285
tsv created 8286
tsv created 8287
tsv created 8288
tsv created 8289
tsv created 8290
tsv created 8291
tsv created 8292
tsv created 8293
tsv created 8294
tsv created 8295
tsv created 8296
tsv created 8297
tsv created 8298
tsv created 8299
tsv created 8300
tsv created 8301
tsv created 8302
tsv created 8303
tsv created 8304
tsv created 8305
tsv created 8306
tsv created 8307
tsv created 8308
tsv created 8309
tsv created 8310
tsv created 8311
tsv created 8312
tsv created 8313
tsv created 8314
tsv created 8315
tsv created 8316
tsv created 8317
tsv created 8318
tsv created 8319
tsv created 8320
tsv created 8321
tsv created 8322
tsv created 83

tsv created 8747
tsv created 8748
tsv created 8749
tsv created 8750
tsv created 8751
tsv created 8752
tsv created 8753
tsv created 8754
tsv created 8755
tsv created 8756
tsv created 8757
tsv created 8758
tsv created 8759
tsv created 8760
tsv created 8761
tsv created 8762
tsv created 8763
tsv created 8764
tsv created 8765
tsv created 8766
tsv created 8767
tsv created 8768
tsv created 8769
tsv created 8770
tsv created 8771
tsv created 8772
tsv created 8773
tsv created 8774
tsv created 8775
tsv created 8776
tsv created 8777
tsv created 8778
tsv created 8779
tsv created 8780
tsv created 8781
tsv created 8782
tsv created 8783
tsv created 8784
tsv created 8785
tsv created 8786
tsv created 8787
tsv created 8788
tsv created 8789
tsv created 8790
tsv created 8791
tsv created 8792
tsv created 8793
tsv created 8794
tsv created 8795
tsv created 8796
tsv created 8797
tsv created 8798
tsv created 8799
tsv created 8800
tsv created 8801
tsv created 8802
tsv created 8803
tsv created 8804
tsv created 88

tsv created 9229
tsv created 9230
tsv created 9231
tsv created 9232
tsv created 9233
tsv created 9234
tsv created 9235
tsv created 9236
tsv created 9237
tsv created 9238
tsv created 9239
tsv created 9240
tsv created 9241
tsv created 9242
tsv created 9243
tsv created 9244
tsv created 9245
tsv created 9246
tsv created 9247
tsv created 9248
tsv created 9249
tsv created 9250
tsv created 9251
tsv created 9252
tsv created 9253
tsv created 9254
tsv created 9255
tsv created 9256
tsv created 9257
tsv created 9258
tsv created 9259
tsv created 9260
tsv created 9261
tsv created 9262
tsv created 9263
tsv created 9264
tsv created 9265
tsv created 9266
tsv created 9267
tsv created 9268
tsv created 9269
tsv created 9270
tsv created 9271
tsv created 9272
tsv created 9273
tsv created 9274
tsv created 9275
tsv created 9276
tsv created 9277
tsv created 9278
tsv created 9279
tsv created 9280
tsv created 9281
tsv created 9282
tsv created 9283
tsv created 9284
tsv created 9285
tsv created 9286
tsv created 92

tsv created 9711
tsv created 9712
tsv created 9713
tsv created 9714
tsv created 9715
tsv created 9716
tsv created 9717
tsv created 9718
tsv created 9719
tsv created 9720
tsv created 9721
tsv created 9722
tsv created 9723
tsv created 9724
tsv created 9725
tsv created 9726
tsv created 9727
tsv created 9728
tsv created 9729
tsv created 9730
tsv created 9731
tsv created 9732
tsv created 9733
tsv created 9734
tsv created 9735
tsv created 9736
tsv created 9737
tsv created 9738
tsv created 9739
tsv created 9740
tsv created 9741
tsv created 9742
tsv created 9743
tsv created 9744
tsv created 9745
tsv created 9746
tsv created 9747
tsv created 9748
tsv created 9749
tsv created 9750
tsv created 9751
tsv created 9752
tsv created 9753
tsv created 9754
tsv created 9755
tsv created 9756
tsv created 9757
tsv created 9758
tsv created 9759
tsv created 9760
tsv created 9761
tsv created 9762
tsv created 9763
tsv created 9764
tsv created 9765
tsv created 9766
tsv created 9767
tsv created 9768
tsv created 97